In [2]:
gene=Array.new #Creating an empty array for saving each line of the document
File.foreach("documents/ArabidopsisSubNetwork_GeneList.txt"){ |line|  gene <<line } #Saving each line of the document
                                                      # in the array. Source: https://www.rubyguides.com/2015/05/working-with-files-ruby/
gene_p=Array.new   #Creating an empty array for saving procesed name of genes (it has metacharacter \n)
gene.each do |line| 
  gene_p << line.gsub("\n",'') #Eliminating metacharacter \n in each line. 
end   #The difference between gene and gene_p array is that in the second one we can access to each value gene_p[1]
     # and the return has not metacharacter \n

["AT4g27030\n", "AT5g54270\n", "AT1g21400\n", "AT5g19120\n", "AT2g13360\n", "AT4g05180\n", "AT1g22690\n", "AT2g45170\n", "AT4g09650\n", "AT5g55620\n", "AT1g31330\n", "AT1g80440\n", "AT3g28270\n", "AT4g12800\n", "AT5g04140\n", "AT5g64040\n", "AT1g29660\n", "AT1g15820\n", "AT1g64500\n", "AT1g03130\n", "AT2g20670\n", "AT4g17090\n", "AT1g23740\n", "AT2g46340\n", "AT4g17460\n", "AT1g07010\n", "AT1g32220\n", "AT2g03750\n", "AT3g47470\n", "AT4g12830\n", "AT5g05690\n", "AT5g64410\n", "AT2g21330\n", "AT1g15980\n", "AT2g01890\n", "AT1g12250\n", "AT2g21210\n", "AT4g28660\n", "AT1g52230\n", "AT3g18890\n", "AT4g22890\n", "AT1g11850\n", "AT1g37130\n", "AT2g04039\n", "AT3g48420\n", "AT4g19170\n", "AT5g07020\n", "AT5g65010\n", "AT3g01500\n", "AT1g25230\n", "AT2g34430\n", "AT1g18060\n", "AT2g30600\n", "AT4g33010\n", "AT1g65490\n", "AT3g21670\n", "AT4g27260\n", "AT1g12780\n", "AT1g55480\n", "AT2g17880\n", "AT3g49160\n", "AT4g21210\n", "AT5g14740\n", "AT4g37980\n", "AT1g29395\n", "AT2g39730\n", "AT1g2002

In [3]:
gene[1]

"AT5g54270\n"

In [4]:
gene_p[2]

"AT1g21400"

In [57]:
require 'rest-client'   # This access to the page of each gene (it works but it is too slow)

#We have to search in other data base the interactions

gene_p.each do |accessor|
  address = "https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=#{accessor}&style=raw"

  response = RestClient::Request.execute(
  method: :get,
  url: address)
  record=response.body
  puts record.match(/.*Uniprot.*/)
end

FT                   /db_xref="Uniprot/SWISSPROT:Q9SZ42"
FT                   /db_xref="Uniprot/SPTREMBL:F4K0A7"
FT                   /db_xref="Uniprot/SPTREMBL:A0A1P8AMS6"
FT                   /db_xref="Uniprot/SPTREMBL:A0A654G2T9"
FT                   /db_xref="Uniprot/SWISSPROT:Q56YA5"
FT                   /db_xref="Uniprot/SWISSPROT:Q41932"
FT                   /db_xref="Uniprot/SWISSPROT:Q8GWK5"
FT                   /db_xref="Uniprot/SWISSPROT:Q8S926"
FT                   /db_xref="Uniprot/SWISSPROT:Q9SSS9"
FT                   /db_xref="Uniprot/SPTREMBL:A0A178UNB2"
FT                   /db_xref="Uniprot/SWISSPROT:Q9SHE8"
FT                   /db_xref="Uniprot/SWISSPROT:Q9M8L2"
FT                   /db_xref="Uniprot/SWISSPROT:Q9LHD9"
FT                   /db_xref="Uniprot/SWISSPROT:Q9SUI4"
FT                   /db_xref="Uniprot/SPTREMBL:A0A1P8BBW3"
FT                   /db_xref="Uniprot/SWISSPROT:Q8L7V3"
FT                   /db_xref="Uniprot/SWISSPROT:Q9C7N5"
FT                  

Interrupt: 

Esta linea de abajo es para ir probando con un unico gen

In [64]:
address = "https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=AT4g27030&style=default"

response = RestClient::Request.execute(
method: :get,
url: address)  
puts response


ID   4    standard; DNA; HTG; 1258 BP.
XX
AC   chromosome:TAIR10:4:13571813:13573070:1
XX
SV   chromosome:TAIR10:4:13571813:13573070:1
XX
DT   17-NOV-2022
XX
DE   Arabidopsis thaliana chromosome 4 TAIR10 partial sequence
DE   13571813..13573070 annotated by Araport11
XX
KW   .
XX
OS   Arabidopsis thaliana (thale-cress)
OC   Eukaryota; Viridiplantae; Streptophyta; Streptophytina; Embryophyta;
OC   Tracheophyta; Euphyllophyta; Spermatophyta; Magnoliopsida; Mesangiospermae;
OC   eudicotyledons; Gunneridae; Pentapetalae; rosids; malvids; Brassicales;
OC   Brassicaceae; Camelineae.
XX
CC   This sequence displays annotation from Ensembl Genomes, based on underlying
CC   annotation from Araport11 ( https://araport.org ). See
CC   http://www.ensemblgenomes.org for more information.
XX
CC   All feature locations are relative to the first (5') base of the sequence
CC   in this file.  The sequence presented is always the forward strand of the
CC   assembly. Features that lie outside of the sequen

In [73]:
record=response.body
puts record.match(/\/.*Uniprot.*/)[0]


/db_xref="Uniprot/SWISSPROT:Q9SZ42"


www.ebi.ac.uk/intact/ws/interactor  BASE URL


it needs that we convert genes id into Uniprot
It provides UNiprot gene A interacts with UNiprot gene B